In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from netCDF4 import Dataset
import os
from mpl_toolkits.basemap import Basemap, shiftgrid

# Drake Passage Transport
First let's like at the 20 year timeseries of Drake Passage transport from our two comparison runs. For this we will use tx_trans, summed across Drake Passage.


In [2]:
datadir = '/g/data/v45/mom01_comparison/'
expt = 'KDS75'
expdir = os.path.join(datadir,expt)

plt.figure(figsize=(10,6))

time = []
DPTrans = []
    
outList = [f for f in os.listdir(expdir) if f.startswith('output')]
outList.sort()

for g in outList:
    oceanfile = os.path.join(expdir,g,'ocean.nc')
    ncFile = Dataset(oceanfile)
    time = np.append(time,ncFile.variables['time'][:]/365. + 45.) 
    tx_trans = ncFile.variables['tx_trans'][0,:,225:650,2100:2101]
    ncFile.close()

    DPTrans = np.append(DPTrans,np.ma.sum(tx_trans))
    
plt.plot(time,DPTrans,label=expt,linewidth=2)

expt = 'GFDL50'
expdir = os.path.join(datadir,expt)

time = []
DPTrans = []
    
outList = [f for f in os.listdir(expdir) if f.startswith('output')]
outList.sort()

for g in outList[17:]:
    oceanfile = os.path.join(expdir,g,'ocean.nc')
    #print oceanfile
    ncFile = Dataset(oceanfile)
    time = np.append(time,ncFile.variables['time'][:]/365.) 
    tx_trans = ncFile.variables['tx_trans'][0,:,225:650,2100:2101]
    ncFile.close()

    DPTrans = np.append(DPTrans,np.ma.sum(tx_trans))
    
plt.plot(time,DPTrans,label=expt,linewidth=2)

plt.legend(loc=4)
plt.ylabel('Drake Passage Transport (Sv)')
plt.xlabel('Time (years)')
plt.title('Drake Passage Transport')
plt.savefig('figures/DPTrans.pdf')

KeyboardInterrupt: 

# Scalar Quantities
Look at some quantities that are saved in ocean_scalar.nc.

In [ ]:
datadir = '/g/data/v45/mom01_comparison/'
plt.figure(figsize=(12,4))

expt = 'KDS75'
expdir = os.path.join(datadir,expt)

time=[];
ke_tot = [];
temp_global_ave = [];

outList = [f for f in os.listdir(expdir) if f.startswith('output')]
outList.sort()

ii=0
for g in outList:
    ii+=1
    oceanfile = os.path.join(expdir,g,'ocean_scalar.nc')
    ncFile = Dataset(oceanfile) 
    time = np.append(time,ncFile.variables['time'][:]/365. + 45.) 
    ke_tot = np.append(ke_tot,ncFile.variables['ke_tot'][:]) 
    temp_global_ave = np.append(temp_global_ave,ncFile.variables['temp_global_ave'][:]) 
    ncFile.close()


plt.subplot(121)
plt.plot(time,ke_tot,label=expt,linewidth=2)

plt.subplot(122)
plt.plot(time,temp_global_ave,label=expt,linewidth=2)

expt = 'GFDL50'
expdir = os.path.join(datadir,expt)

time=[];
ke_tot = [];
temp_global_ave = [];

outList = [f for f in os.listdir(expdir) if f.startswith('output')]
outList.sort()

ii=0
for g in outList:
    ii+=1
    oceanfile = os.path.join(expdir,g,'ocean_scalar.nc')
    ncFile = Dataset(oceanfile) 
    time = np.append(time,ncFile.variables['time'][:]/365.) 
    ke_tot = np.append(ke_tot,ncFile.variables['ke_tot'][:]) 
    temp_global_ave = np.append(temp_global_ave,ncFile.variables['temp_global_ave'][:]) 
    ncFile.close()

plt.subplot(121)
plt.plot(time,ke_tot,label=expt,linewidth=2)


plt.subplot(122)
plt.plot(time,temp_global_ave,label=expt,linewidth=2)

plt.subplot(121)
plt.legend(loc=4)
plt.ylabel('Total KE (10$^{15}$ J)')
plt.xlabel('Time (years)')
plt.title('Global Average KE')

plt.subplot(122)
plt.legend(loc=4)
plt.ylabel('Global average temp ($^o$C)')
plt.xlabel('Time (years)')
plt.title('Global Average Temperature')

plt.savefig('figures/Scalars.pdf')

# Overturning Circulation
Next, let's look at overturning circulation in density space using ty_trans_rho. We will zonally average this diagnostic, without accounting for the tripolar grid, so ignore the Arctic.

In [ ]:
datadir = '/g/data/v45/mom01_comparison/'
plt.figure(figsize=(12,12))

expt = 'KDS75'
expdir = os.path.join(datadir,expt)

outList = [f for f in os.listdir(expdir) if f.startswith('output')]
outList.sort()

ii=0
for g in outList[-24:]: 
    ii+=1
    oceanfile = os.path.join(expdir,g,'ocean.nc')
    #print oceanfile
    ncFile = Dataset(oceanfile) 
    potrho = ncFile.variables['potrho'][:]
    yu = ncFile.variables['grid_yu_ocean'][:]
    ty_trans_rho = ncFile.variables['ty_trans_rho'][0,:,:,:]
    ncFile.close()

    if ii==1:
        psi = np.ma.sum(np.ma.cumsum(ty_trans_rho,axis=0),axis=2)
    else:
        psi = psi + np.ma.sum(np.ma.cumsum(ty_trans_rho,axis=0),axis=2)
        
psi_KDS = psi/ii

#print psi_KDS

#x_formatter = matplotlib.ticker.ScalarFormatter(useOffset=False)
clev = np.arange(-20,20,2)

print "Final file was ", oceanfile

plt.subplot(311)
plt.contourf(yu,potrho,psi_KDS,cmap=plt.cm.PiYG,levels=clev,extend='both')
cb=plt.colorbar(orientation='vertical',shrink = 0.7)
cb.ax.set_xlabel('Sv')
plt.contour(yu,potrho,psi_KDS,levels=clev,colors='k',linewidths=0.25)
plt.contour(yu,potrho,psi_KDS,levels=[0.0,],colors='k',linewidths=0.5)
plt.gca().invert_yaxis()
#plt.gca().yaxis.set_major_formatter(x_formatter)
plt.ylim((1037.5,1034))
plt.ylabel('Potential Density (kg m$^{-3}$)')
plt.xlabel('Latitude ($^\circ$N)')
plt.xlim([-75,85])
plt.title('Overturning in KDS75')


expt = 'GFDL50'
expdir = os.path.join(datadir,expt)

outList = [f for f in os.listdir(expdir) if f.startswith('output')]
outList.sort()

ii=0
for g in outList[-20:]:
    ii+=1
    oceanfile = os.path.join(expdir,g,'ocean.nc')
    #print oceanfile
    ncFile = Dataset(oceanfile) 
    potrho = ncFile.variables['potrho'][:]
    yu = ncFile.variables['grid_yu_ocean'][:]
    ty_trans_rho = ncFile.variables['ty_trans_rho'][0,:,:,:]
    ncFile.close()

    if ii==1:
        psi = np.ma.sum(np.ma.cumsum(ty_trans_rho,axis=0),axis=2)
    else:
        psi = psi + np.ma.sum(np.ma.cumsum(ty_trans_rho,axis=0),axis=2)
        
psi_GFDL = psi/ii

print "Final file was ", oceanfile

#x_formatter = matplotlib.ticker.ScalarFormatter(useOffset=False)
clev = np.arange(-20,20,2)

plt.subplot(312)
plt.contourf(yu,potrho,psi_GFDL,cmap=plt.cm.PiYG,levels=clev,extend='both')
cb=plt.colorbar(orientation='vertical',shrink = 0.7)
cb.ax.set_xlabel('Sv')
plt.contour(yu,potrho,psi_GFDL,levels=clev,colors='k',linewidths=0.25)
plt.contour(yu,potrho,psi_GFDL,levels=[0.0,],colors='k',linewidths=0.5)
plt.gca().invert_yaxis()
#plt.gca().yaxis.set_major_formatter(x_formatter)
plt.ylim((1037.5,1034))
plt.ylabel('Potential Density (kg m$^{-3}$)')
plt.xlabel('Latitude ($^\circ$N)')
plt.xlim([-75,85])
plt.title('Overturning in GFDL50')

psi_diff = psi_KDS - psi_GFDL
dlev = np.arange(-5,5,0.5)

plt.subplot(313)
plt.contourf(yu,potrho,psi_diff,cmap=plt.cm.PuOr,levels=dlev,extend='both')
cb=plt.colorbar(orientation='vertical',shrink = 0.7)
cb.ax.set_xlabel('Sv')
plt.contour(yu,potrho,psi_GFDL,levels=clev,colors='k',linewidths=0.25)
plt.contour(yu,potrho,psi_GFDL,levels=[0.0,],colors='k',linewidths=0.5)
plt.gca().invert_yaxis()
#plt.gca().yaxis.set_major_formatter(x_formatter)
plt.ylim((1037.5,1034))
plt.ylabel('Potential Density (kg m$^{-3}$)')
plt.xlabel('Latitude ($^\circ$N)')
plt.xlim([-75,85])
plt.title('Overturning: KDS75 - GFDL50')

plt.savefig('figures/Overturning.pdf')

In [ ]:
print 'For densities exceeding: ', potrho[68]
print 'And at latitude:', yu[500]
print np.min(psi_GFDL[68:,500])
print np.min(psi_KDS[68:,500])